# Модуль 14: Оптимизация формул
#tableau
**Тема: Создание эффективных формул в Tableau
Цель**: Научить писать быстрые и надёжные формулы, находить ошибки и работать с большими данными, чтобы дашборды не тормозили.

## Теория
**Что такое оптимизация формул?**
Оптимизация — это когда ты пишешь формулы так, чтобы они работали быстро, не ломались и не грузили систему, особенно на больших данных. Это как готовить суп: можно накидать всё подряд, а можно выбрать правильные ингредиенты и сварить быстро. В Tableau медленные формулы замедляют дашборд, а ошибки в них могут испортить анализ.
**Основные аспекты**  
* **Оптимизация производительности**: Упрощай формулы, чтобы они меньше нагружали Tableau.  
* **Отладка**: Проверяй формулы на ошибки, чтобы результаты были правильными.  
* **Обработка больших данных**: Используй подходы, которые работают с миллионами строк без тормозов.

⠀**Как оптимизировать производительность?**  
* **Избегай сложных вычислений**: Используй простые функции вроде SUM или AVG вместо громоздких IF или SCRIPT_*.  
* **Фильтруй данные раньше**: Применяй фильтры в Data Source или на уровне базы, чтобы уменьшить объём данных до расчётов.  
* **Используй агрегацию**: Работай с агрегированными данными (например, SUM([Sales])), а не с сырыми строками.  
* **Заменяй LOD на простые решения**: Если задача решается табличными вычислениями или фильтрами, не пиши LOD.  
* **Ограничивай R/Python**: Используй SCRIPT_* только для задач, которые Tableau не может решить.

⠀**Как отлаживать формулы?**  
* Создай таблицу в дашборде и перетащи формулу, чтобы увидеть результаты.  
* Разбей сложную формулу на части: сначала проверь каждую часть отдельно.  
* Используй комментарии (// или /* */) для пояснений.  
* Проверь NULL: Используй ZN() или ISNULL() для обработки пропусков.  
* Смотри на Performance Recorder (Help → Settings and Performance → Start Performance Recording), чтобы найти медленные формулы.

⠀**Как работать с большими данными?**  
* **Extract вместо Live**: Создай Extract в Tableau, чтобы данные хранились локально и обрабатывались быстрее.  
* **Агрегируй в базе**: Используй SQL для предобработки данных перед загрузкой в Tableau.  
* **Ограничивай строки**: Добавляй фильтры в Data Source (например, [Year] = 2025), чтобы загружать меньше данных.  
* **Используй индексы**: Если работаешь с базой, убедись, что нужные поля проиндексированы.  
* **Тестируй на выборке**: Проверяй формулы на малом наборе данных, прежде чем применять к миллионам строк.

⠀**Как фильтры влияют на формулы?**  
* **Фильтры в рабочем листе**: Применяются *до* расчёта формул в дашборде. Если фильтр убирает данные (например, [Region] = "West"), формула работает только с ними.  
* **Фильтры на уровне Data Source**: Убирают данные ещё до загрузки в Tableau, уменьшая объём для формул. Например, фильтр [Order Date] = 2025 ограничивает все расчёты.  
* **Фильтры табличных вычислений**: Если формула использует RANK, фильтр табличного вычисления может показать только часть результата (например, топ-3).  
* **Совет**: Проверяй фильтры в Data Source, чтобы знать, какие данные участвуют в формуле.

⠀**Сравнение оптимизированных и неоптимизированных формул**  
| **Что сравниваем** | **Оптимизированные формулы** | **Неоптимизированные формулы** |
|:-:|:-:|:-:|
| **Скорость** | Быстрые, используют агрегацию и фильтры в базе. | Медленные, с кучей IF, LOD или SCRIPT_* на сырых данных. |
| **Читаемость** | Простые, с комментариями и без лишних условий. | Сложные, без пояснений, трудно понять. |
| **Работа с большими данными** | Работают с миллионами строк благодаря Extract и SQL. | Тормозят на больших данных из-за обработки строк. |
| **Пример** | SUM([Sales]) с фильтром в Data Source. | IF ISNULL([Sales]) THEN 0 ELSE [Sales] END без фильтров. |

**Почему это важно?**
Оптимизированные формулы — это как хорошо настроенный двигатель: дашборд работает быстро, не ломается и справляется с большими данными. Отладка помогает избежать ошибок, а работа с большими данными делает твои анализы масштабируемыми. Это экономит время и делает отчёты надёжными, даже если данных миллионы.

## Примеры с таблицами
**Исходная таблица базы данных** (Superstore):  
| Order ID | Category | Region | Customer Name | Order Date | Sales | Profit |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1 | Technology | West | Anna | 2025-01-01 | 1000 | 200 |
| 2 | Technology | East | Anna | 2025-02-25 | 2000 | 400 |
| 3 | Furniture | West | Bob | 2025-01-15 | 1500 | 300 |
| 4 | Furniture | East | Bob | 2025-03-01 | 500 | -50 |
| 5 | Technology | West | Charlie | 2025-01-10 | 800 | 100 |
* **Оптимизация: Упрощение формулы для доли продаж**  
  * **Условие**: Хотим посчитать долю продаж каждой категории от общего итога.  
  * **Что делаем**:  
    * Вместо неоптимизированной формулы:  
```
IF ISNULL(SUM([Sales])) THEN 0 ELSE SUM([Sales]) / SUM({FIXED : SUM([Sales])}) END // Доля продаж (медленная)
```
    **Пишем оптимизированную:** 
```
IF ISNULL(SUM([Sales])) THEN 0 ELSE SUM([Sales]) / SUM({FIXED : SUM([Sales])}) END // Доля продаж (медленная)
```
    **Добавляем фильтр в Data Source**: [Order Date] = 2025.
* **Как работает**: TOTAL быстрее, чем FIXED, а фильтр в Data Source уменьшает объём данных.  
* **Пример**: Общие продажи = 3000 (Technology) + 2000 (Furniture) = 5000.  
**Итоговая таблица** (визуализация с [Category]):  
| Category | SUM([Sales]) | Доля продаж |
|:-:|:-:|:-:|
| Technology | 3000 | 60% |
| Furniture | 2000 | 40% |
**Что получаем**: Быстрая формула с долей продаж.

* **Отладка: Проверка формулы для топ-2 клиентов**  
  * **Условие**: Хотим показать топ-2 клиентов по продажам, но формула даёт ошибку.  
  * **Что делаем**:  
    * Ошибочная формула:   IF RANK(SUM([Sales])) <= 2 THEN SUM([Sales]) END // Топ-2 клиентов (ошибка)
    * Исправляем:   IF RANK(SUM([Sales])) <= 2 THEN SUM([Sales]) ELSE 0 END // Топ-2 клиентов
    * Разбиваем на части: создаём отдельное поле RANK(SUM([Sales])), проверяем в таблице.  
    * Тестируем в дашборде с [Customer Name].
  * **Как работает**: Добавляем ELSE 0, чтобы избежать NULL, и проверяем ранг отдельно.  
  * **Пример**: Anna (3000), Bob (2000), Charlie (800).  
**Итоговая таблица** (визуализация с [Customer Name]):  
| Customer Name | SUM([Sales]) | Топ-2 клиентов |
|:-:|:-:|:-:|
| Anna | 3000 | 3000 |
| Bob | 2000 | 2000 |
| Charlie | 800 | 0 |
**Что получаем**: Исправленная формула показывает продажи топ-2 клиентов.

* **Большие данные: Агрегация в SQL**  
  * **Условие**: Хотим посчитать средние продажи по регионам на большом датасете (миллионы строк).  
  * **Что делаем**:  
    * Вместо обработки в Tableau пишем Custom SQL в Data Source:  
    SELECT Region, AVG(Sales) as Avg_Sales
    FROM Superstore
    WHERE Order_Date = '2025'
    GROUP BY Region
    * Создаём Extract для ускорения.
  * **Как работает**: SQL агрегирует данные в базе, а Extract хранит их локально.  
  * **Пример**: West (1000, 1500, 800), East (2000, 500).  
**Итоговая таблица** (визуализация с [Region]):  
| Region | Avg_Sales |
|:-:|:-:|
| West | 1100 |
| East | 1250 |
**Что получаем**: Быстрый расчёт средних продаж без тормозов.
* **Оптимизация: Замена LOD на фильтр**  
  * **Условие**: Хотим показать продажи только для категорий с общей прибылью > 300.  
  * **Что делаем**:  
    * Неоптимизированная формула:   IF {FIXED [Category] : SUM([Profit])} > 300 THEN SUM([Sales]) ELSE 0 END // Продажи прибыльных категорий
    * Оптимизированная:  
      * Создаём набор "Прибыльные категории" (Condition: SUM([Profit]) > 300).  
      * Используем формулу:   IF [Прибыльные категории] THEN SUM([Sales]) ELSE 0 END // Продажи прибыльных категорий
  * **Как работает**: Набор быстрее LOD, так как не требует пересчёта для каждого значения.  
  * **Пример**: Technology (Profit: 600), Furniture (Profit: 250).  
**Итоговая таблица** (визуализация с [Category]):  
| Category | SUM([Sales]) | Продажи прибыльных категорий |
|:-:|:-:|:-:|
| Technology | 3000 | 3000 |
| Furniture | 2000 | 0 |
**Что получаем**: Быстрая формула, Furniture обнулена.

* **Отладка: Обработка NULL в формуле**  
  * **Условие**: Хотим посчитать прибыль на заказ, но некоторые заказы имеют NULL.  
  * **Что делаем**:  
    * Ошибочная формула:   SUM([Profit]) / COUNT([Order ID]) // Прибыль на заказ
    * Исправляем:   ZN(SUM([Profit])) / COUNT([Order ID]) // Прибыль на заказ
    * Проверяем в таблице с [Customer Name].
  * **Как работает**: ZN заменяет NULL на 0, избегая ошибок.  
  * **Пример**: Anna (600/2), Bob (250/2), Charlie (100/1).  
**Итоговая таблица** (визуализация с [Customer Name]):  
| Customer Name | Прибыль на заказ |
|:-:|:-:|
| Anna | 300 |
| Bob | 125 |
| Charlie | 100 |
**Что получаем**: Корректная прибыль на заказ без ошибок.
* **Большие данные: Использование Extract и фильтров**  
  * **Условие**: Хотим показать топ-3 региона по продажам на большом датасете.  
  * **Что делаем**:  
    * Добавляем фильтр в Data Source: [Order Date] = 2025.  
    * Создаём Extract.  
    * Пишем формулу:   IF RANK(SUM([Sales])) <= 3 THEN SUM([Sales]) ELSE 0 END // Топ-3 регионов
    * Настраиваем область вычислений: Table (Down).
  * **Как работает**: Фильтр уменьшает данные, Extract ускоряет обработку, а формула работает с агрегированными значениями.  
  * **Пример**: West (3300), East (2500).  
**Итоговая таблица** (визуализация с [Region]):  
| Region | SUM([Sales]) | Топ-3 регионов |
|:-:|:-:|:-:|
| West | 3300 | 3300 |
| East | 2500 | 2500 |
**Что получаем**: Быстрый расчёт топ-2 регионов (в примере только два региона).
⠀
## Рекомендации для новичков
* **Думай о скорости**: Задавай фильтры в Data Source и используй агрегацию (например, SUM([Sales])), чтобы уменьшить объём данных.  
* **Тестируй в таблице**: Построй таблицу с [Category] или [Region], добавь формулу и проверь результаты.  
* **Разбивай формулы**: Если формула сложная, создавай промежуточные вычисляемые поля для проверки.  
* **Проверяй фильтры**: Фильтры в рабочем листе и Data Source влияют на формулы. Data Source фильтры (например, [Year] = 2025) ограничивают всё.  
* **Обрабатывай NULL**: Пиши ZN(SUM([Sales])) или ISNULL([Sales], 0) для пропусков.  
* **Пиши комментарии**: Добавляй // Доля продаж или /* Топ-3 регионов */ в формулы, чтобы не запутаться.  
* **Используй Extract**: Для больших данных создавай Extract, чтобы дашборд не тормозил.  
* **Не усложняй**: Если можно решить фильтром, набором или простым SUM, не пиши LOD или SCRIPT_*.  
* **Смотри Performance Recorder**: Запусти его, чтобы найти медленные формулы (Help → Start Performance Recording).

## Почему это важно?
Оптимизированные формулы — это как лёгкий рюкзак в походе: дашборд работает быстро, не падает и справляется с любым объёмом данных. Отладка спасает от ошибок, а работа с большими данными делает твои анализы масштабируемыми. Освоишь оптимизацию — твои дашборды будут летать, а пользователи скажут тебе спасибо за скорость и точность.